<center>
    <h1> ILI-286 - Laboratorio #1 </h1>
    <h2> Computación numérica de vectores propios aplicados a PCA </h2>
</center>

| Nombre | Rol | Email |
| :----- | :-- | :---- |
| Marco Rojas | 201073005-0 | marco.rojaso@alumnos.usm.cl |
| Hernán Vargas | 201073009-3 | hernan.vargas@alumnos.usm.cl |


## Tabla de contenidos
* [Introducción](#intro)
* [Desarrollo y analisís de resultados:](#desarrollo)
 1. [Power Iteration y Rayleigh Quotient](#de1)
 2. [Naive k-first eigen finder](#de2)
 3. [Clever k-first eigen finder](#de3)
 4. [TODO](#de4)
 5. [Power iteration v/s Rayleigh Quotient iteration](#de5)
 6. [Naive v/s Clever](#de6)
 7. [Todos los valores y vectores propios](#de7)
* [Concluciones](#Concluciones)
* [Referencias](#Referencias)

<div id='intro' />
## Introducción
Escribir algo aquí.

<div id='desarrollo'/>
## Desarrollo y analisís de resultados

Debemos comenzar cargando las bibliotecas y datos necesarios. Además se definirán algunas variables que se usarán más adelante.

In [ ]:
import numpy as np
#Para utilizar %memit
#%load_ext memory_profiler

dataset = np.load("arcene.npy")
sigma_x = np.dot((1/(1-dataset.shape[1]))*np.transpose(dataset), dataset)

def normalize(A):
    return np.divide(A,(np.linalg.norm(A)))

Además como se calcularán las complejidades de varios algoritmos se debe tener en cuenta lo siguiente:

La multiplicación de una matriz $A_{n\times m}$ por $B_{m\times p}$ tiene complejidad $O(nmp)$. En este caso trabajamos con matrices cuadradas y vectores así que nuestra complejidad del producto matriz - vector será: $O(n^2)$

La normalización de una matriz $A_{n\times m}$ incluye una multiplicación de matriz por escalar y una división por la norma de una matriz. 
La multiplicación y la división tendrá complejidad $O(n^2)$ para las matrices $A_{n\times n}$.
La norma que consiste en elevar cada elemento ($O(n^2)$), sumarlos ($O(n^2)$) y sacar la raíz cuadráda ($\sim O(n)$) de esta suma. Así la norma tambien tendrá complejidad de $O(n^2)$ y por ello el proceso de normalización tendrá complejidad de $O(n^2)$

Además podemos notar que la complejidad de calcular la transpuesta de $A_{n\times n}$ será $O(n^2)$

Por último, la complejidad de `numpy.linalg.solve` es de $O(n^3)$ pues utiliza el algorítmo *LAPACK*[[1](#ref1)] podemos ver su complejidad en [[2](#ref2)]

<div id='de1'/>
### 1.- Power Iteration y Rayleigh Quotient

Los siguientes algoritmos requieren una matriz $A_{n\times n}$.
Opcionalmente pueden recibir el vector inicial $x_{n\times 1}$ (por defecto $[1 \dots 1]^{T}$) y un número máximo de iteraciones (por defecto $1000$).

Ambos algoritmos retornan una tupla con el valor y vector propio dominante: $(\lambda, \vec{v})$

In [ ]:
def power_iteration(A, x=None, max_iter=1000):
    #Comprueba las dimensiones de x.
    if x is None: x = np.ones([A.shape[0],1])
    elif A.shape[0] != x.shape[0]: raise ValueError("Initial vector error.")
    #Bucle de resolución.
    lamb_old = False;
    for j in range(1, max_iter):
        u = normalize(x)
        x = np.dot(A,u)
        lamb = np.dot(np.transpose(u),x)
        if (lamb == lamb_old):
            break;
        lamb_old = lamb;
    u = normalize(x)
    return lamb[0,0], u

def rayleigh_quotient_iteration(A, x=None, max_iter=1000):
    #Comprueba las dimensiones de x.
    if x is None: x = np.ones([A.shape[0],1])
    elif A.shape[0] != x.shape[0]: raise ValueError("Initial vector error.")
    #Bucle de resolución.
    I = np.identity(A.shape[0])
    lamb_old = False;
    for i in range(1, max_iter):
        u = normalize(x)
        lamb = np.dot(np.dot(np.transpose(u), A), u)
        C = A - lamb*I
        try:
            x = np.linalg.solve(C, u)
        except:
            break 
        if (lamb == lamb_old):
            break;
        lamb_old = lamb;
    u = normalize(x)
    return lamb[0,0], u

#### a) Complejidad algoritmo Power Iteracion: 
En el bucle principal de *Power Iteration* se realiza una normalización, dos productos matriz - vector y una transpuesta, así su complejidad será: $k(O(n^2) + 2O(n^2) + O(n^2)) = O(kn^2)$
     
Como vemos, la complejidad de *Power Iteration* viene dada principalmente por la cantidad de iteraciones $k$ que alcance a realizar antes de converger. Como la tasa de convergencia de este algorítmo es lineal se espera que $k$ sea grande.
     
     
#### b) Complejidad algoritmo Rayleigh Quotient Iteration:
El bucle principal de *Rayleigh Quotient Iteration* efectua una normalización, una transpuesta, dos productos puntos y un *solve*, así su complejidad estará dada por: $k(4O(n^2) + O(n^3)) = O(kn^3)$
     
$k$ al igual que en el caso anterior representa la cantidad de iteraciones que alcanza a realizar el algoritmo, este $k$ será mucho menor (convergencia cuadrática) que en *power iteration* pues el algoritmo hace converger el vector propio más rapidamente especialmente cuando la matris $A$ es simetrica (convergencia cúbica).
 

<div id='de2'/>
### 2.- Naive k-first eigen finder
#### a) Correctitud:

Llamemos $B = A - \lambda_1 v_1 v_1^T$, supongamos $A$ simetrica y por ello sus vectores propios ortogonales. Calcularemos los vectores y valores propios de B:
$$ B v_i = (A - \lambda_1 v_1 v_1^T)v_i$$
$$ B v_i = Av_i - \lambda_1 v_1 v_1^Tv_i$$

Luego, digamos $v_i$ unitario por lo que para $i=1$ tenemos $ v_1^T v_i = 1$ y  para $i\neq 1$ tenemos que $ v_1^T v_i = 0 $ 

Así, $B v_i = \lambda_i v_i $ para $i=2:n$

Es decir, los valores y vectores propios de $B$ con respecto a $A$ son iguales a excepción del primer valor propio de $B$ que es 0 a diferencia de $A$. Entonces podemos usar ésta propiedad para calcular los valores propios dominantes i-ésimos de $A$ con *Power Iteration* u otro algoritmo de búsqueda de valor propio dominante.

#### b) Complejidad del algoritmo kEigenFinder:
En el bucle principal de `kEigenFinder` se realiza una transpuesta, un producto punto, una resta y un *power iteration*,
así, suponiendo que $k_1$ es el número de valores y vectores propios a obtener y $k_2$ es el promedio de iteraciones de *power iteration* tenemos que la complejidad del algorítmo será: $k_1(3O(n^2 + O(k_2n^2))) = O(k_1k_2n^2)$

#### c) Implementación:
`kEigenFinder` toma como argumento una matriz simetrica $A_{n\times n}$. Opcionalmente un número $k$ de valores y vectores propios a obtener (por defecto todos) y un vector $p$ inicial para cada ejecución de *power iteration*.

Retorna un `array` con valores propios y una matriz con vectores propios.

In [ ]:
def kEigenFinder(A, k = None, p = None):
    if p is None: p = np.ones([A.shape[0],1])
    elif A.shape[0] != p.shape[0]: raise ValueError("Initial vector error.")
    if k is None: k = A.shape[0]
    elif k > A.shape[0]: raise ValueError("k is out of range.")
    
    lamb = np.zeros([p.shape[0],1]);
    l = 0;
    v_finales = np.zeros([p.shape[0], p.shape[0]]);
    v = np.zeros([p.shape[0], 1]);
    for j in range(0, k):
        A = A-l*np.dot(v,v.transpose());
        l, v = power_iteration(A, p);
        v_finales[:,[j]] = v;
        lamb[j] = l;
    return lamb, v_finales 
A= np.matrix([[3,4,5],[4,2,1],[5,1,8]])
print(kEigenFinder(A))#, np.matrix([1,1,1]).transpose(), 3))

<div id='de3'/>
### 3.- Clever k-first eigen finder
#### a) Correctitud
#### b) Complejidad algoritmo kEigenFinderPP:
Con una matriz $A_{n\times n}$ tenemos que la complejidad será:
 * $n^2$ por cada producto matriz vector.
 * $2n^2$ por cada normalización.
 * $p*n^2$ por cada potenciación.

Así,la complejidad de kEigenFinderPP para encontrar $k$ valores propios será: 
    $k(4n^2 + 2n^2 + pn^2) ~= O(kpn^2)$

#### c) Implementación

In [ ]:
def kEigenFinderPP(A, p = 5, k = None):
    if k is None: k = A.shape[0]
    elif k > A.shape[0]: raise ValueError("k is out of range.")
    #Listas donde se guardarán los resulados.
    lamb = [None] * k
    v    = [None] * k
    #Valores iniciales.
    A_p  = np.linalg.matrix_power(A, p)
    #x = np.random.random([A.shape[0],1])
    x = np.ones([A.shape[0],1])
    #Bucle de resolución.
    for i in range(0, k):
        x = np.dot(A_p, x)
        v[i] = normalize(x)
        lamb[i]= np.dot(np.dot(np.transpose(v[i]), A), v[i])[0,0]
        A_p = A_p - lamb[i]**p * np.dot(v[i], np.transpose(v[i])) #Esto se ejecuta una vez demás
    return lamb, v

print(A)
#print(B)
l, y = kEigenFinderPP(A,8)
print(l,y)
#print(A)

<div id='de4'/>
### 4.- Modificación a #ToDo.
#### a) ¿Que debemos modificar?
#### b) Complejidad computacional
#### c) Implementación

In [ ]:
def normalized_simultaneous_iteration(A):
    Q = np.identity(A.shape[0]);
    for j in range (1,1000):
        Q, R = np.linalg.qr(np.dot(A,Q))
    return np.diag(np.dot(np.dot(np.transpose(Q), A),Q));

def normalized_simultaneous_iteration_q(A,k):
    Q = np.identity(A.shape[0]);
    Q = Q[:,0:k]
#    Q_old = False;
    for j in range (1,1000):
        Q, R = np.linalg.qr(np.dot(A,Q))
#        if (np.linalg.norm(Q_old-Q, np.inf)<= 10^-1):
#            break;
#        Q_old = Q;
    print(j);
    return np.diag(np.dot(np.dot(np.transpose(Q), A),Q));

A = np.matrix([[3,4,5],
               [4,2,1],
               [5,1,8]]);

print(normalized_simultaneous_iteration(A))
print(normalized_simultaneous_iteration_q(A,1))
print(normalized_simultaneous_iteration_q(A,2))
print(normalized_simultaneous_iteration_q(A,3))

### 5.- Power iteration v/s Rayleigh Quotient iteration.

In [ ]:
#Comentado por carga
#%timeit power_iteration(sigma_x, max_iter=20)
#%memit  power_iteration(sigma_x, max_iter=20)

#%timeit rayleigh_quotient_iteration(sigma_x, max_iter=20)
#%memit  rayleigh_quotient_iteration(sigma_x, max_iter=20)

#TODO: comparacion y concluciones...
<div id='de6'/>
### 6.- Naive v/s Clever v/s

In [ ]:
K = [10, 100, 1000]
P = 10
k = 1
#%timeit kEigenFinder(sigma_x, None, k)
#%memit  kEigenFinder(sigma_x, None, k)

#%timeit kEigenFinder(sigma_x, P, k)
#%memit  kEigenFinder(sigma_x, P, k)

<div id='de7'/>
### 6.- Todos los valores y vectores propios

## Concluciones

## Referencias
<div id='ref1'\> [1] [NumPy v1.10 Manual](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html) numpy.linalg.solve. *The solutions are computed using LAPACK routine _gesv*. Revisado 30/10/2015

<div id='ref2'\> [2] [LAPACK Benchmark](http://www.netlib.org/lapack/lug/node71.html). Ver tabla 3.13. Revisado el 30/10/2015